# Notebook d'association de différents Datasets

A partir de deux datasets mal formalisés, c'est à dire avec des colonnes de texte communes mais pas d'identifiant commun, le but de ce notebook est de tenter de trouver des correspondances afin d'associer les clés. 

Ici, nous avons deux datasets avec chacun :
- une colonne correspondant au nom du musée (de type str)
- une colonne correspondant à la ville du musée (de type str)
- une colonne avec un identifiant, mais les identifiants associés à chaque musée sont différents dans les deux datasets. 

Evidemment, tenter de rapprocher les deux datasets via la colonne des noms de musée n'est pas suffisant :
- Il y a des fautes de frappe et des fautes d'orthographe
- Certains musées n'ont pas exactement la même dénomination même si ce sont les mêmes
    - exemple : *musée departemental...* et *..*
- Problèmes de formalisation des chaines de cractères : 
    - exemple : informatiquement, les mots 'Ecole', 'École', 'école' et 'ecole' sont différents.
    - exemple : 'Saint-Etienne' et 'Saint Etienne' sont différents
    - exemple : ' Musée' et 'Musée' sont différents
    
**Il est donc nécessaires de trouver des méthodes automatiques et rigoureuses afin de trouver les correspondances entre les deux datasets.**

**Nous tenterons tout d'abord des méthodes simples, puis essaierons des methodes liées à des techniques de NLP (Traitement Automatique du Langage), de type TF-IDF et cosine similarity, afin d'afiner les résultats et tenter de trouver le plus grand nombre de correspondances possibles.**

## Imports nécessaires:

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import unidecode

#Necessary imports for tfidf and cosine similarity:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

## Imports des Datasets:

Comme évoqué plus haut, deux datasets sont considérés. 
- Un dataset sur la fréquentation des musées. 
- Un dataset correspondant à la base Muséofile du ministère de la culture. 

Ces deux datasets sont en OpenData, mis à disposition par le ministère de la culture et disponible [ici pour la base portant sur la fréquentation](https://data.culture.gouv.fr/explore/dataset/frequentation-des-musees-de-france/) et [ici pour la base Muséofile](https://data.culture.gouv.fr/explore/dataset/musees-de-france-base-museofile/information/). 

Un nettoyage a été réalisé sur ces données brutes. 

Le notebook correspondant à ce nettoyage peut être trouvé [ici](), et je vous invite à vous y reporter pour des affichages ou pour obtenir des informations supplémentaires sur ces datasets.  

Nous allons importer ces deux datasets via un DataFrame `Pandas`.

In [3]:
#Base des fréquentations:
df_freq=pd.read_csv('data_clean/df_freq_clean.csv')
#Base Muséofile:
df_museofile=pd.read_csv('data_clean/df_geo_museofile.csv')

In [7]:
df_freq.head()

,REF_DU_MUSEE,REGION,NOM_DU_MUSEE,VILLE,Type_de_fréquentation,Année,Entrées
0,7511701,ÎLE-DE-France,Musée National Jean-Jacques Henner,PARIS,Payante,2008,0.0
1,7511701,ÎLE-DE-France,Musée National Jean-Jacques Henner,PARIS,Payante,2002,803.0
2,7511701,ÎLE-DE-France,Musée National Jean-Jacques Henner,PARIS,Payante,2004,0.0
3,7511701,ÎLE-DE-France,Musée National Jean-Jacques Henner,PARIS,Payante,2010,8729.0
4,7511801,ÎLE-DE-France,Musée de Montmartre,PARIS,Payante,2009,50541.0


In [8]:
df_museofile.head(2)

,Identifiant,Adresse,Artiste,Atout,Catégorie,Code_Postal,Domaine_thématique,Département,Date_de_saisie,Histoire,...,Région,Téléphone,Thèmes,URL,Ville,geolocalisation,coord_x,coord_y,merc_x,merc_y
0,M0410,"Parc de Bécon, 178 Boulevard Saint Denis","Ary Scheffer, René Ménard, Fernand Roybet, Car...",Situé dans le parc de Bécon dont l’origine rem...,Maison d'artiste,92400,Arts décoratifs;Art moderne et contemporain;Be...,Hauts-de-Seine,2019-01-21,Les collections furent constituées au départ p...,...,Ile-de-France,01 71 05 77 92,"Beaux-Arts : Dessin, Estampe et Affiche, Peint...",http://www.museeroybetfould.fr,Courbevoie,"48.900998,2.271279",48.900998,2.271279,252837.621729,6.258079e+06
1,M5076,Place Monsenergue,"Joseph Vernet, Pascal de La Rose, Morel-Fatio,...",Conservatoire de l'histoire de l'arsenal et du...,NaN,83000,Histoire,Var,2019-01-21,Créé à la fin du Premier Empire et ouvert au p...,...,Provence-Alpes-Côte d'Azur,04 94 02 02 01,"Archéologie nationale : Gallo-romain, Moderne;...",http://www.musee-marine.fr,Toulon,"43.122018,5.929368",43.122018,5.929368,660054.226486,5.330563e+06


## Fonctions de nettoyage :

La fonction suivante, `format_string`, a pour but de formaliser de façon automatique les chaines de caractères à considérer.

Plusieurs étapes de nettoyage et de formalisation sont réalisées dans cette fonction:
- `strip`: permet de supprimer les espaces au début et à la fin de la chaine de caractères, s'il y en a
    - exemple : ' Musée' sera transformé en 'Musée'
- `lower`: permet de mettre toute la chaine de caractères en minuscules 
    - exemple : 'Musée' sera transformé en 'musée'
- Méthode `unidecode` : permet d'enlever les accents 
    - exemple : 'musée' sera transformé en 'musee'
- Remplacement des `-` et des `'` par des espaces
    - exemple : 'musee-historique' sera transformé en 'musee historique'
    - exemple : 
- Suppression des espaces multiples

In [4]:
def format_string(string):
    """
    function 'format_string' to transform a given string 
    
    @Inputs : 
    -string, str
    
    @Outputs : 
    -string, str
    """
    string=unidecode.unidecode(string.strip().lower())
    string=string.replace('-',' ')
    string=string.replace('\'', " ")
    return re.sub(' +', ' ', string)

### Exemples d'utilisation:

In [5]:
format_string(" Musée d'art")

'musee d art'

In [6]:
format_string("Musée      des Beaux-Arts")

'musee des beaux arts'

### Application:

D'une part, sur le DataFrame `df_freq`, nous allons appliquer la fonction `format_string` sur les colonnes suivantes : 
- `NOM_DU_MUSEE`
- `VILLE`

D'autre part, sur le DataFrame `df_museofile`, nous allons de même appliquer la fonction `format_string` sur les colonnes suivantes :

- `Nom_officiel`
- `Ville`

Ainsi, nous allons pouvoir ainsi extraire deux dataframes *de travail*, comportant pour chaque dataframe l'ID, le nom et la ville du musée.

#### Sur la base des fréquentations

In [10]:
df_freq['NOM_DU_MUSEE']=df_freq['NOM_DU_MUSEE'].apply(format_string)
df_freq['VILLE']=df_freq['VILLE'].apply(format_string)

On créé un DataFrame de travail, `df_freq_travail` avec les reférences des musées, les noms et les villes. 

On veut une seule ligne par `REF DU MUSEE`.

In [11]:
df_freq_travail=df_freq.groupby('REF_DU_MUSEE').first().reset_index()[['REF_DU_MUSEE','NOM_DU_MUSEE','VILLE']]

#### Sur la base Museofile

On applique là aussi des transformations pour formatter le texte (nom du musée et ville)

In [14]:
df_museofile['Nom_officiel']=df_museofile['Nom_officiel'].astype(str).apply(format_string)
df_museofile['Ville']=df_museofile['Ville'].astype(str).apply(format_string)

On créé de la même façon qu'avec la base des fréquentations, une base de travail pour museofile:

In [15]:
df_museofile_travail=df_museofile[['Identifiant','Nom_officiel','Ville']].copy()

#### Visualisation des deux Datasets:

In [13]:
df_freq_travail.head()

,REF_DU_MUSEE,NOM_DU_MUSEE,VILLE
0,0105301,musee du brou,bourg en bresse
1,0105302,musee departemental des pays de l ain,bourg en bresse
2,0106401,musee de la societe d histoire et d archeologie,briord
3,0113701,musee departemental du revermont,val revermont
4,0119201,musee archeologique,izernore


In [16]:
df_museofile_travail.head()

,Identifiant,Nom_officiel,Ville
0,M0410,musee roybet fould,courbevoie
1,M5076,musee national de la marine,toulon
2,M5071,musee des plans reliefs,paris
3,M0387,musee de la batellerie,conflans sainte honorine
4,M0449,musee du colombier,ales


Les deux datasets comportent bien les colonnes que nous voulions, et les valeurs de celles-ci semblent bien avoir été nettoyées.

#### Vérification du nombre d'identifiant pour l'unicité :

In [20]:
print("Nombre de ligne dans le DataFrame Museofile : ",df_museofile_travail.shape[0])
print("Nombre d'Identifiant dans le DataFrame Museofile : ", df_museofile_travail['Identifiant'].nunique())

Nombre de ligne dans le DataFrame museofile :  1114
Nombre d'Identifiant dans la base Museofile :  1114


Il y a bien un identifiant par la ligne dans le DataFrame `df_museofile_travail`.

In [22]:
print("Nombre de ligne dans le DataFrame des fréquentations : ",df_freq_travail.shape[0])
print("Nombre d'Identifiant dans le DataFrame des fréquentations : ", df_freq_travail['REF_DU_MUSEE'].nunique())

Nombre de ligne dans le DataFrame des fréquentations :  1249
Nombre d'Identifiant dans le DataFrame des fréquentations :  1249


Il y a bien un identifiant par ligne dans le DataFrame `df_freq_travail`


**Par conséquent, il y a donc bien unicité des identifiants pour les deux dataframes.**

On va donc essayer, plus loin, d'associer la colonne `REF DU MUSEE` avec la colonne `Identifiant` de la base Museofile, via les colonnes correspondant aux noms des musées et de la ville de ceux-ci.

#### Vérification des Nans:

In [23]:
df_museofile_travail.isna().sum()

Identifiant     0
Nom_officiel    0
Ville           0
dtype: int64

In [24]:
df_freq_travail.isna().sum()

REF_DU_MUSEE    0
NOM_DU_MUSEE    0
VILLE           0
dtype: int64

**Il n'y a pas de nans dans ces deux datasets de travail.**

## Application du rapprochement entre les deux dataframes simplement avec ce nettoyage/ cette transformation.

### Application de la jointure

Nous allons simplement utiliser la méthode `merge` de Pandas, avec pour arguements:

Mise en place de cela:

In [27]:
df_freq_museofile=pd.merge(left=df_museofile_travail,
                           right=df_freq_travail,
                           left_on=['Nom_officiel', 'Ville'],
                           right_on=['NOM_DU_MUSEE', 'VILLE'],
                           how='left')

Vérification du nombre d'éléments (qui doit être égal au nombre de lignes présents dans le DataFrame `df_museofile_travail`

In [30]:
print('Nombre de lignes dans le Dataframe df_museofile_travail : ', df_museofile_travail.shape[0])
print('Nombre de lignes dans le Dataframe df_freq_travail : ', df_freq_travail.shape[0])
print('Nombre de lignes dans le Dataframe fusionné : ', df_freq_museofile.shape[0])

Nombre de lignes dans le Dataframe df_museofile_travail :  1114
Nombre de lignes dans le Dataframe df_freq_travail :  1249
Nombre de lignes dans le Dataframe fusionné :  1114


Le comportement de la jointure entre les deux DataFrame est donc bien celui que nous voulions, avec un nombre de ligne dans la jointure égal au nombre de lignes du DataFrame `df_museofile_travail`.

### Résultats:

In [31]:
df_freq_museofile.isna().sum()

Identifiant       0
Nom_officiel      0
Ville             0
REF_DU_MUSEE    450
NOM_DU_MUSEE    450
VILLE           450
dtype: int64

In [32]:
1114-450

664

**664 musées ont bien été trouvés.**

Cependant, **450** musées de la base Museofile n'ont pas été trouvés dans la base des fréquentations. 

Nous allons donc observer les résultats plus en détail afin d'essayer de trouver des pistes d'amélioration.

#### Observation détaillée des résultats.

In [33]:
df_freq_museofile.head()

,Identifiant,Nom_officiel,Ville,REF_DU_MUSEE,NOM_DU_MUSEE,VILLE
0,M0410,musee roybet fould,courbevoie,9202601,musee roybet fould,courbevoie
1,M5076,musee national de la marine,toulon,NaN,NaN,NaN
2,M5071,musee des plans reliefs,paris,NaN,NaN,NaN
3,M0387,musee de la batellerie,conflans sainte honorine,NaN,NaN,NaN
4,M0449,musee du colombier,ales,3000702,musee du colombier,ales


In [35]:
df_freq_travail[df_freq_travail['VILLE'].str.startswith('conflans')]

,REF_DU_MUSEE,NOM_DU_MUSEE,VILLE
1041,7817201,musee de la batellerie et des voies navigables,conflans sainte honorine


**We can clearly see that there are some inclusive property (but not equality) betweens the two dataframes.**

## Méthodes pour tester l'inclusivité entre les colonnes des DataFrames: 

### Explication

En prenant le même exemple que précédemment, au lieu de tester l'exact égalité entre deux strings, on peut tester l'inclusivité. 

In [40]:
'musee de la batellerie' in 'musee de la batellerie et des voies navigables'

True

Ainsi, deux conditions pour que deux musées soient considérés comme étant les mêmes:
- Le nom du musée est contenu dans le nom de l'autre

**et**

- La ville est contenue dans la ville de l'autre

### Fonction :

In [123]:
def merge_inclusivity(df_join, df_right, col_left, col_right, other_col_left, other_col_right):
    
    #Copy of the df_join:
    df_join_copy=df_join.copy()
    columns_to_update=df_right.columns
    #Loop over the nans elements: 
    for index_na, element_na in df_join[df_join[columns_to_update[1]].isna()].iterrows():
        for index_right, element_right in df_right.iterrows():
            #If condition
            if (((element_na[col_left] in element_right[col_right])
            or (element_right[col_right] in element_na[col_left]))
                
            and ((element_na[other_col_left] in element_right[other_col_right])
            or (element_right[other_col_right] in element_na[other_col_left]))):
                
                #Updating columns:
                for col_up in columns_to_update:
                    df_join_copy[col_up].iloc[index_na]=element_right[col_up]
                    
    return df_join_copy

### Application 

Il faut définir les arguments de la fonction écrite ci-dessus:

In [124]:
df_join=df_freq_museofile
df_right=df_freq_travail
col_left='Nom_officiel'
col_right='NOM_DU_MUSEE'
other_col_left='Ville'
other_col_right='VILLE'

On applique la fonction `merge_inclusivity`:

In [125]:
df_freq_museofile=merge_inclusivity(df_join, df_right, col_left, col_right, other_col_left, other_col_right)

### Résultats

In [126]:
df_freq_museofile.isna().sum()

Identifiant       0
Nom_officiel      0
Ville             0
REF_DU_MUSEE    155
NOM_DU_MUSEE    155
VILLE           155
dtype: int64

A l'étape précédente, il restait **450** musées à trouver. 

Il en reste maintenant **155**. 

Nous avons donc trouvé **295** musées rien qu'en testant l'inclusivité. 

#### Observation détaillée des résultats:

In [127]:
df_freq_museofile.head()

,Identifiant,Nom_officiel,Ville,REF_DU_MUSEE,NOM_DU_MUSEE,VILLE
0,M0410,musee roybet fould,courbevoie,9202601,musee roybet fould,courbevoie
1,M5076,musee national de la marine,toulon,8313704,musee national de la marine de toulon,toulon
2,M5071,musee des plans reliefs,paris,NaN,NaN,NaN
3,M0387,musee de la batellerie,conflans sainte honorine,7817201,musee de la batellerie et des voies navigables,conflans sainte honorine
4,M0449,musee du colombier,ales,3000702,musee du colombier,ales


Nous pouvons tout d'abord observer que l'exemple développé ci-dessus a maintenant été trouvé comme prévu.

#### Observation des musées restant à trouver :

In [55]:
df_freq_museofile[df_freq_museofile['REF_DU_MUSEE'].isna()].head()

,Identifiant,Nom_officiel,Ville,REF_DU_MUSEE,NOM_DU_MUSEE,VILLE
2,M5071,musee des plans reliefs,paris,NaN,NaN,NaN
9,M0526,musee de la guerre de 1870,gravelotte,NaN,NaN,NaN
24,M0370,musee de l ecole de barbizon auberge ganne,barbizon,NaN,NaN,NaN
25,M1063,musee departemental alexandre franconie,cayenne,NaN,NaN,NaN
33,M0857,musee departemental gassendi,digne les bains,NaN,NaN,NaN


In [57]:
df_freq_travail[df_freq_travail['VILLE'].str.startswith('gravelotte')]

,REF_DU_MUSEE,NOM_DU_MUSEE,VILLE
673,5725601,musee departemental de la guerre de 70,gravelotte


Nous pouvons voir ici que ...

In [58]:
df_freq_travail[df_freq_travail['VILLE'].str.startswith('cayenne')]

,REF_DU_MUSEE,NOM_DU_MUSEE,VILLE
1241,9730201,musee territorial alexandre franconie,cayenne guyane
1242,9730901,musee des cultures guyanaises,cayenne


Ici aussi, 

### Conclusions sur la méthode d'inclusivité:

Comme nous l'avons vu, considérer l'inclusivité entre les colonnes a permis d'améliorer significativement les résultats. 

Cependant, cela ne suffit pas. 

Il faut donc trouver des méthodes plus sophistiquées pour améliorer encore nos résultats. 

Ces méthodes, développées en détail ci-après, concernent l'utilisation d'outils de NLP(Natural Language Processing - Traitement Automatique du Langage) et TF-IDF et cosine similarity. 

## Méthodes plus complexes avec TF-IDF : 

**Ce que l'on veut faire:**
- Prendre le DataFrame avec seulement les éléments non trouvés avant. 
- Tester toutes les combinaisons d'un DataFrame de base, sur le deuxième DataFrame que l'on veut associer au premier. 
- Renvoyer le nombre de combinaisons trouvées
- Renvoyer la liste entière des combinaisons trouvées pour pouvoir vérifier. 


**Deux méthodes à tester:**
- Ne prendre que le nom du musée pour tfidf
    - Trouver les noms les plus proches
    - Prendre le nom le plus proche et vérifier qua la ville est la même
- Prendre le nom du musée plus la ville pour tfidf
    - Trouver les noms concaténés avec la ville les plus proches (au sens cosine similarity)
    - Prendre le nom le plus proche et vérifier que la ville est la même

**Pour ces deux méthodes:**
- Associer le score de similarité dans le nouveau DataFrame
    - Trier par score croissant (on peut estimer qu'à partir du moment où on n'a plus d'erreurs tout le reste est bon, mais il est tout de même important de pouvoir vérifier
- Threshold sur le score de similarité ? 
- La fonction le nouveau DataFrame, avec les éléments non trouvés dans un premier temps. 

### Première méthode : Ne considérer que le nom du musée dans la tf-idf

#### Explication :

#### Fonction :

In [165]:
def find_return_answers(df_missing_left, df_right, column_left,
                        column_right, other_column_left, other_column_right, top_n=5):
    """
    This function, from one dataset with nans, and the other to be associated with the first one, 
    returns a new dataFrame with what has been found
    
    @Inputs:
    -df_missing_left
    -df_right
    -column_left
    -column_right
    -Other_column_left
    -Other_column_right
    -top_n, int, optional,the number of top_n answers to be retreived
    @Outputs:
    -df_new_values
    """
    #Initializing a tfidf model:
    tfidf = TfidfVectorizer(sublinear_tf=True, min_df=0, norm='l2', encoding='latin-1', ngram_range=(1,1))
    #Applying the new tfidf model on the right Dataframe and on the column specified:
    tfidf_matrix_right = tfidf.fit_transform(df_right[column_right]).toarray()
    #Creating the DataFrame this function is going to be returning:
    #Firstly, this DataFrame countains only the uncomplete rows:
    df_new_values=df_missing_left.copy()
    #We add the sim_score column to save the similarity scores:
    df_new_values['sim_score']=0
    #Loop over the indexes and elements in the new DataFrame:
    for index,element in df_new_values.iterrows():
        #Defining the query for each row:
        query=element[column_left]
        #Transforming the query from the tfidf_model:
        tfidf_query=tfidf.transform([query]).toarray()
        #Calculating the cosine similarities between the query and the tfidf_matrix_right
        cosine_similarities = cosine_similarity(tfidf_query,tfidf_matrix_right).flatten()
        #We sort the documents from the similarity to the query :
        related_docs_indices = [i for i in cosine_similarities.argsort()[::-1]]
        #We retrieve the top n documents most similar to the given query :
        indexes_scores = [(index_t, cosine_similarities[index]) for index_t in related_docs_indices][0:top_n]
        #Loop over the index and scores of the answers:
        for index_ans, score_ans in indexes_scores:
            #Verifying over the other column:
            if ((element[other_column_left] in df_right.iloc[index_ans][other_column_right])
                or 
                (df_right.iloc[index_ans][other_column_right] in element[other_column_left])):
                
        
                
                #This answer is then satisfying our conditions, so we can merge the two rows.                
                df_new_values.loc[index]=df_new_values.loc[index].fillna(df_right.iloc[index_ans])
                #We add, for information, the score:
                df_new_values.loc[index,'sim_score']=score_ans
                #We break the loop since we found an answer:
                break
    return df_new_values

#### Application :

On doit tout d'abord définir les arguments que la fonction doit prendre en entrée:

In [166]:
df_missing_left=df_freq_museofile[df_freq_museofile['REF_DU_MUSEE'].isna()].copy()
df_right=df_freq_travail
column_left='Nom_officiel'
column_right='NOM_DU_MUSEE'
other_column_left='Ville'
other_column_right='VILLE'

On peut ensuite appliquer la fonction:

On utilise **top_n=5**

In [167]:
df_answers_test_1=find_return_answers(df_missing_left, df_right, column_left,column_right, other_column_left, other_column_right, top_n=5)

#### Résultats:

Taille du DataFrame retourné par la fonction (composé seulement des éléments non trouvés précédemment)

In [168]:
df_answers_test_1.shape

(155, 7)

Nombre de musées non trouvés pour l'instant:

In [169]:
df_answers_test_1.isna().sum()

Identifiant      0
Nom_officiel     0
Ville            0
REF_DU_MUSEE    29
NOM_DU_MUSEE    29
VILLE           29
sim_score        0
dtype: int64

On peut voir qu'il n'en reste plus que 29 sur les 155 au départ.

In [170]:
df_answers_test_1

,Identifiant,Nom_officiel,Ville,REF_DU_MUSEE,NOM_DU_MUSEE,VILLE,sim_score
2,M5071,musee des plans reliefs,paris,7510702,musee des plans et reliefs,paris,0.011674
9,M0526,musee de la guerre de 1870,gravelotte,5725601,musee departemental de la guerre de 70,gravelotte,0.015373
24,M0370,musee de l ecole de barbizon auberge ganne,barbizon,7702201,musee departemental de l ecole de barbizon aub...,barbizon,0.008186
25,M1063,musee departemental alexandre franconie,cayenne,9730201,musee territorial alexandre franconie,cayenne guyane,0.013014
33,M0857,musee departemental gassendi,digne les bains,0407001,musee gassendi,digne les bains,0.013713
34,M1080,musee au fil du papier,pont a mousson,5443101,"musee ""au fil du papier""",pont a mousson,0.039218
38,M0324,musee palais fesch,ajaccio,2A00403,musee fesch,ajaccio,0.016631
39,M0617,musee de flandre,cassel,5913501,musee departemental de flandre,cassel,0.021228
42,M0925,musee du vieux saint chamas,saint chamas,NaN,NaN,NaN,0.000000
52,M0817,musee departemental de la vie rurale,saint riquier,NaN,NaN,NaN,0.000000


Au vu de ces résultats, la nouvelle fonction semble fonctionner correctement (aucune erreur n'est visible ci-dessus). 

##### Observation détaillée des résultats:

In [86]:
df_answers_test_1[df_answers_test_1['REF_DU_MUSEE'].isna()].tail()

,Identifiant,Nom_officiel,Ville,REF_DU_MUSEE,NOM_DU_MUSEE,VILLE
965,M7018,museum des volcans,aurillac,NaN,NaN,NaN
968,M0118,musee de la ferblanterie,saint arcons d allier,NaN,NaN,NaN
993,M1209,muma musee de mayotte,dzaoudzi,NaN,NaN,NaN
1007,M0109,musee des arts africains et asiatiques (maison...,vichy,NaN,NaN,NaN
1062,M0829,musee capon,marans,NaN,NaN,NaN


In [87]:
df_freq_travail[df_freq_travail['VILLE'].str.startswith('marans')]

,REF_DU_MUSEE,NOM_DU_MUSEE,VILLE
216,1721801,musee cappon,marans


#### Conclusion sur la première méthode :

Même si cette méthode a permis de trouver beaucoup d'éléments, il semble y avoir une limite: les fautes d'orthographe. 

Ainsi, un musée, situé à `Marans`, a été orthographié `'musee cappon'` dans un DataFrame et `'musee capon'` dans un autre. 

Il serait par conséquent sans doute judicieux d'essayer d'établir la tfidf sur la concaténation des colonnes utiles. 

Cela est développé ci-après dans le cadre de la deuxième méthode considérée avec TF-IDF.

### Seconde méthode: considérer une concaténation des colonnes communes dans la TF-IDF

#### Explication

On a pu voir qu'il serait sans doute plus opportun d'ajouter les villes dans la construction des query/matrices TFIDF. 

Nous allons donc essayer et comparer les résultats avec les précédents. 

- Les conditions sur la ville sont elles toujours d'actualité ? 
    - **Oui** 
        - *exemple* : `Musée d'histoire de Paris` à Neuilly et `Musée d'histoire` à Paris
- Threshold ? 

- Ajouter le score pour pouvoir trier du moins bon au meilleur. 

#### Fonction

In [107]:
def find_return_answers_deux(df_missing_left, df_right, column_left,
                        column_right, other_column_left, other_column_right, top_n=5):
    """
    This function, from one dataset with nans, and the other to be associated with the first one, 
    returns a new dataFrame with what has been found
    
    @Inputs:
    -df_missing_left
    -df_right
    -column_left
    -column_right
    -Other_column_left
    -Other_column_right
    -top_n, int, optional,the number of top_n answers to be retreived
    @Outputs:
    -df_new_values
    """
    #Initializing a tfidf model:
    tfidf = TfidfVectorizer(sublinear_tf=True, min_df=0, norm='l2', encoding='latin-1', ngram_range=(1,1))
    #Applying the new tfidf model on the right Dataframe and on the column specified:
    tfidf_matrix_right = tfidf.fit_transform(df_right[column_right].str.cat(df_right[other_column_right].values.astype(str), sep=' ').astype(str)).toarray()
    #Creating the DataFrame this function is going to be returning:
    #Firstly, this DataFrame countains only the uncomplete rows:
    df_new_values=df_missing_left.copy()
    #We add a column representing the similarity score:
    df_new_values['sim_score']=0
    
    #Loop over the indexes and elements in the new DataFrame:
    for index,element in df_new_values.iterrows():
        #Defining the query for each row:
        query=element[column_left]+' '+element[other_column_left]
        #Transforming the query from the tfidf_model:
        tfidf_query=tfidf.transform([query]).toarray()
        #Calculating the cosine similarities between the query and the tfidf_matrix_right
        cosine_similarities = cosine_similarity(tfidf_query,tfidf_matrix_right).flatten()
        #We sort the documents from the similarity to the query :
        related_docs_indices = [i for i in cosine_similarities.argsort()[::-1]]
        #We retrieve the top n documents most similar to the given query :
        indexes_scores = [(index_t, cosine_similarities[index]) for index_t in related_docs_indices][0:top_n]
        #Loop over the index and scores of the answers:
        for index_ans, score_ans in indexes_scores:
            #Verifying over the other column:
            if ((element[other_column_left] in df_right.iloc[index_ans][other_column_right]) 
                or 
                (df_right.iloc[index_ans][other_column_right] in element[other_column_left])):
                
                #This answer is then satisfying our conditions, so we can merge the two rows.                
                df_new_values.loc[index]=df_new_values.loc[index].fillna(df_right.iloc[index_ans])
                #We add, for information, the score:
                df_new_values.loc[index,'sim_score']=score_ans
                #We break the loop since we found an answer:
                break
                
    return df_new_values

#### Application

On doit tout d'abord définir les arguments que la fonction doit prendre en entrée:

In [108]:
df_missing_left=df_freq_museofile[df_freq_museofile['REF_DU_MUSEE'].isna()].copy()
df_right=df_freq_travail
column_left='Nom_officiel'
column_right='NOM_DU_MUSEE'
other_column_left='Ville'
other_column_right='VILLE'

On peut ensuite appliquer la fonction:

On utilise **top_n=5**

In [115]:
df_answers_test_2=find_return_answers_deux(df_missing_left, df_right, column_left,column_right, other_column_left, other_column_right, top_n=5)

#### Résultats

Taille du DataFrame retourné par la fonction (composé seulement des éléments non trouvés précédemment)

In [116]:
df_answers_test_2.shape

(155, 7)

Nombre de musées non trouvés pour l'instant:

In [117]:
df_answers_test_2.isna().sum()

Identifiant     0
Nom_officiel    0
Ville           0
REF_DU_MUSEE    5
NOM_DU_MUSEE    5
VILLE           5
sim_score       0
dtype: int64

On peut voir qu'il n'en reste plus que 5 sur les 155 au départ.

In [118]:
df_answers_test_2

,Identifiant,Nom_officiel,Ville,REF_DU_MUSEE,NOM_DU_MUSEE,VILLE,sim_score
2,M5071,musee des plans reliefs,paris,7510702,musee des plans et reliefs,paris,0.008716
9,M0526,musee de la guerre de 1870,gravelotte,5725601,musee departemental de la guerre de 70,gravelotte,0.044543
24,M0370,musee de l ecole de barbizon auberge ganne,barbizon,7702201,musee departemental de l ecole de barbizon aub...,barbizon,0.005433
25,M1063,musee departemental alexandre franconie,cayenne,9730201,musee territorial alexandre franconie,cayenne guyane,0.007084
33,M0857,musee departemental gassendi,digne les bains,0407001,musee gassendi,digne les bains,0.007567
34,M1080,musee au fil du papier,pont a mousson,5443101,"musee ""au fil du papier""",pont a mousson,0.028664
38,M0324,musee palais fesch,ajaccio,2A00403,musee fesch,ajaccio,0.010057
39,M0617,musee de flandre,cassel,5913501,musee departemental de flandre,cassel,0.009065
42,M0925,musee du vieux saint chamas,saint chamas,1309201,musee municipal paul lafran,saint chamas,0.006018
52,M0817,musee departemental de la vie rurale,saint riquier,NaN,NaN,NaN,0.000000


Au vu de ces résultats, la nouvelle fonction semble fonctionner correctement (aucune erreur n'est visible ci-dessus). 

##### Observation détaillée des résultats:

En reprenant l'exemple ci-dessus:

In [113]:
df_answers_test_2[df_answers_test_2['Ville'].str.startswith('marans')]

,Identifiant,Nom_officiel,Ville,REF_DU_MUSEE,NOM_DU_MUSEE,VILLE,sim_score
1062,M0829,musee capon,marans,1721801,musee cappon,marans,0.011882


L'exemple de mauvaise association developpée dans la première méthode a ici été trouvé.

Vérifions les musées non trouvés plus haut mais trouvés ici:

In [114]:
df_answers_test_2[df_answers_test_1['REF_DU_MUSEE'].isna()]

,Identifiant,Nom_officiel,Ville,REF_DU_MUSEE,NOM_DU_MUSEE,VILLE,sim_score
42,M0925,musee du vieux saint chamas,saint chamas,1309201,musee municipal paul lafran,saint chamas,0.006018
52,M0817,musee departemental de la vie rurale,saint riquier,NaN,NaN,NaN,0.000000
81,M0108,musee saint marc,souvigny,0327501,musee municipal de souvigny,souvigny,0.009722
167,M0993,musee hydrelec,vaujany,3800501,conservatoire musee des industries hydroelectr...,vaujany,0.011887
232,M0834,musee oleronais alienor d aquitaine,saint pierre d oleron,NaN,NaN,NaN,0.000000
240,M0597,"musee du protestantisme, de la reforme a la la...",fontrieu,NaN,NaN,NaN,0.000000
247,M0989,musee de la prehistorie du vercors,vassieux en vercors,2636401,musee du site prehistorique,vassieux en vercors,0.004661
310,M0318,musee de l affiche,chaumont,5212101,musee des beaux arts et d archeologie,chaumont,0.052378
360,M0329,musee d ethnographie corse,bastia,2B03301,musee de bastia,bastia cedex,0.006783
368,M0036,musee francais du chemin de fer,mulhouse,NaN,NaN,NaN,0.000000


**BEAUCOUP D'ERREURS!!**

#### Conclusion sur la seconde méthode

Certains résultats ont pu être retrouvés comme nous le voulions, mais d'autres semblent faux. 

Cela est un problème qui va devoir être résolu.

### Conclusion sur l'utilisation de TF-IDF et Cosine Similarity

## Outil de suppression des résultats non satisfaisants:

In [173]:
a=input()

['bla', 'bli', 'blo']


In [174]:
import ast

In [176]:
list_a=ast.literal_eval(a)

In [177]:
list_a[0]

'bla'

In [ ]:
def delete_bad_results(df, list_a):
    for Id_bad in list_a:
        df.drop[]

## Conclusion sur les résultats généraux : 

Tout d'abord, la méthode très simple de nettoyage des termes a premis de trouver un grand nombre de musées. 

Ensuite, considérer non plus l'égalité parfaite les inclusions des termes, a permis là aussi d'améliorer grandement les résultats. 

Enfin, deux méthodes issues de TFIDF ont été considérés et mises en place. 
- Le première, en ne comparant qu'une seule colonne. 

- La seconde en comparant (au sens de TF-IDF et de cosine similarity) la concaténation du nom du musée et de la ville, c'ets à dire de deux colonnes. 

La première méthode a permis d'augmenter les performances, en trouvant plus de musées. 
La seconde a permis de trouver beaucoup de musées mais s'est aussi trompée sur un certain nombre d'exemples.

Par conséquent, introduire la possibilité pour l'utilisateur de regarder quels sont les résultats qui sont inexactes pour les supprimer de la liste finale des résultats pourrait être une bonne idée, et a été développé en dernier lieu. 

Afin de ne pas surcharger le présent notebook, un autre notebook, intitulé Linking_Datasets_auto et disponible dans le repo GitHub ou ici, permet de développer une fonction globale pour développer toute cette démarche de façon totalement automatique. 